In [2]:
import os,sys #user to create and modify file name and save it
from netCDF4 import Dataset#to read the .nc files
import pandas as pd #for dataframe activities
from datetime import datetime,timedelta #for the time related activities
import matplotlib.pyplot as plt #for the Ploting purpose
import numpy as np #numerical fucntions
import datetime 
import warnings #to ignore unnecessary warnings
import xarray as xr
import numpy as np
import regionmask
import geopandas as gp #ReadingShapefile
import glob
from cartopy import config
from pyhdf.SD import SD, SDC
# from zmq import ctx_opt_names #for the Ploting purpose
warnings.filterwarnings('ignore')

In [3]:

from scipy.interpolate import griddata
min_lon = 78.00
max_lon = 96.00
min_lat = 8.00
max_lat = 23.00

xi=np.arange(min_lon,max_lon+1,1)
yi=np.arange(min_lat,max_lat+1,1)

## for Vetical Velocity

In [27]:

os.chdir('/Volumes/PtatoBasket/Datasets/ERA5/VerticalVelocity/')
#Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])

# PreMon
file='PreMonDaily_VerticalVelocity.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
wi=ds.w
mask=countries_mask_poly.mask(wi,lat_name='latitude',lon_name='longitude')
masked_shape=wi.where(mask==0)
masked_shape=masked_shape.interp(latitude=yi).interp(longitude=xi)
masked_shape.to_netcdf('FPreMonDaily_VerticalVelocity1.nc')

# Winter
file='WinterDaily_VerticalVelocity.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
wi=ds.w[1:,:,:]
masked_shape=masked_shape.where(mask==0)
masked_shape=masked_shape.interp(latitude=yi).interp(longitude=xi)
masked_shape.to_netcdf('FWinterDaily_VerticalVelocity1.nc')

## for LTS
<img  width=400 src="../formulas/LTS.png">
<br>
<img  width=400 src="../formulas/pot_temp.png">


Calculating LTS

In [ ]:
os.chdir('/Volumes/PtatoBasket/Datasets/ERA5/LTS/')

# PreMon
file='PreMonDaily_T700.nc'
ds=xr.open_dataset(file)
t700=ds.t
theta700=t700*np.power((1000/700),.286)

file='PreMonDaily_T1000.nc'
ds=xr.open_dataset(file)
theta1000=ds.t

LTS_Premon=theta700-theta1000
LTS_Premon.to_netcdf('PreMonDaily_LTS.nc')


# Winter
file='WinterDaily_T700.nc'
ds=xr.open_dataset(file)
t700=ds.t
theta700=t700*np.power((1000/700),.286)

file='WinterDaily_T1000.nc'
ds=xr.open_dataset(file)
theta1000=ds.t

LTS_Premon=theta700-theta1000
LTS_Premon.to_netcdf('WinterDaily_LTS.nc')

Regridding and clipping

In [28]:

os.chdir('/Volumes/PtatoBasket/Datasets/ERA5/LTS/')
#Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])

# PreMon
file='PreMonDaily_LTS.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
wi=ds.t
mask=countries_mask_poly.mask(wi,lat_name='latitude',lon_name='longitude')
masked_shape=wi.where(mask==0)
masked_shape=masked_shape.interp(latitude=yi).interp(longitude=xi)
masked_shape.to_netcdf('FPreMonDaily_LTS.nc')

# Winter
file='WinterDaily_LTS.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
wi=ds.t[1:,:,:]
masked_shape=wi.where(mask==0)
wi=masked_shape.interp(latitude=yi).interp(longitude=xi)
wi.to_netcdf('FWinterDaily_LTS.nc')

## For RH/Uwind850/Vwind850

In [29]:

os.chdir('/Volumes/PtatoBasket/Datasets/ERA5/RH750/')
#Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])

# PreMon
file='PreMonDaily_RH750.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
wi=ds.r
mask=countries_mask_poly.mask(wi,lat_name='latitude',lon_name='longitude')
masked_shape=wi.where(mask==0)
wi=masked_shape.interp(latitude=yi).interp(longitude=xi)
wi.to_netcdf('FPreMonDaily_RH750.nc')

# Winter
file='WinterDaily_RH750.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
w=ds.r[1:,:,:]
masked_shape=wi.where(mask==0)
wi=masked_shape.interp(latitude=yi).interp(longitude=xi)
wi.to_netcdf('FWinterDaily_RH750.nc')

### For MeanWind and WindDirection

In [6]:


os.chdir('/Volumes/PtatoBasket/Datasets/ERA5/')
file='/Volumes/PtatoBasket/Datasets/ERA5/U-Wind/FPreMonDaily_Uwind850.nc'
ds=xr.open_dataset(file)
u=ds.u

file='/Volumes/PtatoBasket/Datasets/ERA5/V-Wind/FPreMonDaily_Vwind850.nc'
ds=xr.open_dataset(file)
v=ds.v
# Compute the wind speed
wind_speed = np.sqrt(u**2 + v**2)

# Compute the wind direction (in degrees)
wind_dir = np.degrees(np.arctan2(u, v))

# The wind direction is measured counterclockwise from east,
# so if you want it to be measured clockwise from north,
# you can add 360 to negative values and take the modulo 360.
wind_dir = (wind_dir + 360) % 360

wind_speed.to_netcdf('FPreMonDaily_MeanWind850.nc')
wind_dir.to_netcdf('FPreMonDaily_WindDir850.nc')

### For RH-averaged

In [5]:

# PreMon
os.chdir('/Volumes/PtatoBasket/Dataset_New/ERA-5 RH/PreMon')
file='PreMonDaily_RHavg.nc'
ds=xr.open_dataset(file)
ds=ds.reindex(latitude=list(reversed(ds.latitude)))
wi=ds.r
wi=wi.interp(latitude=yi).interp(longitude=xi)
wi.to_netcdf('FPreMonDaily_RHavg.nc')


## For MERRA2 AOD and AI

In [30]:

os.chdir('/Volumes/PtatoBasket/Datasets/MERRA2/')
#Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])

# PreMon
file='MERRA-PreMon.nc'
wi=xr.open_dataset(file)
mask=countries_mask_poly.mask(wi,lat_name='lat',lon_name='lon')
masked_shape=wi.where(mask==0)
masked_shape=masked_shape.interp(lat=yi).interp(lon=xi)
masked_shape.to_netcdf('FPreMonDaily_AODAI.nc')

# Winter
file='MERRA-Winter.nc'
wi=xr.open_dataset(file)
mask=countries_mask_poly.mask(wi,lat_name='lat',lon_name='lon')
masked_shape=wi.where(mask==0)
masked_shape=masked_shape.interp(lat=yi).interp(lon=xi)
masked_shape.to_netcdf('FWinterDaily_AODAI.nc')


### For Making a group with smoke and dust aerosol groups

In [16]:
os.chdir('/Volumes/PtatoBasket/Datasets/MERRA2/')
file='Composites/N_MERRA-PreMon.nc'
ds=xr.open_dataset(file)
ds=ds.interp(lat=yi).interp(lon=xi)

sm=ds.BC_AOD+ds.OC_AOD+ds.SULFATE_AOD
dst=ds.SEASALT_AOD+ds.DUST_AOD

sm.to_netcdf('FPreMonDaily_SMOKEAOD.nc')
dst.to_netcdf('FPreMonDaily_DUSTAOD.nc')



### For PBLH

NPreparing dataset is done in a temporary code same as that of RH

In [10]:
os.chdir('/Volumes/PtatoBasket/Dataset_New/PBLH')
file='PreMonDaily_PBLH.nc'
ds=xr.open_dataset(file)
ds=ds.interp(lat=yi).interp(lon=xi)
ds.to_netcdf('FPreMonDaily_PBLH.nc')


# FOR IMGERG Precipitation

In [12]:
os.chdir('/Volumes/PtatoBasket/Dataset_New/IMERG/')
file='PreMonDaily_IMGERG.nc4'
ds=xr.open_dataset(file)

#Creating Mask
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])
mask=countries_mask_poly.mask(ds,lat_name='lat',lon_name='lon')

masked_shape=ds.where(mask==0)
ds=masked_shape.interp(lat=yi).interp(lon=xi)
precip=ds.precipitationCal

precip.to_netcdf('FPreMonDaily_Precip.nc')

# For MODIS

In [34]:

os.chdir('/Volumes/PtatoBasket/Datasets/MODIS/')
#Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])

# PreMon
file='FINAL_MODIS_PreMon.nc'
wi=xr.open_dataset(file)
mask=countries_mask_poly.mask(wi,lat_name='latitude',lon_name='longitude')
masked_shape=wi.where(mask==0)
masked_shape=masked_shape.interp(latitude=yi).interp(longitude=xi)
masked_shape.to_netcdf('FPreMonDaily_4MODIS.nc')

# Winter
file='FINAL_MODIS_Winter.nc'
wi=xr.open_dataset(file)
mask=countries_mask_poly.mask(wi,lat_name='latitude',lon_name='longitude')
masked_shape=wi.where(mask==0)
masked_shape=masked_shape.interp(latitude=yi).interp(longitude=xi)
masked_shape.to_netcdf('FWinterDaily_4MODIS.nc')


## For Precipitable water

In [14]:
os.chdir('/Volumes/PtatoBasket/Dataset_New/PrecipitableWater-MODIS/')
file='PreMonDaily_PrecipWater.nc'
ds=xr.open_dataset(file)
ds=ds.interp(latitude=yi).interp(longitude=xi)
ds.to_netcdf('FPreMonDaily_PrecipWater.nc')